In [1]:
from functools import partial
import keras

In [ ]:
# Belangrijk op examen: hoe werkt een partial?
# Partial laat je toe om een callable te instantiëren met standaard parameters.
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

# Eerste argument: functie of iets dat je kan aanroepen. -> je krijgt de constructor (tf.keras.layers.Conv2D in voorbeeld)

# Met volgende argumenten gaan we de defaults aanpassen en is DefaultConv2D eigenlijk gelijk aan
keras.layers.Conv2D(kernel_size=3, strides=1, padding="same",
                    kernel_initializer="he_normal", use_bias=False)

# Als we dan later één van die parameters willen aanpassen, moeten we niet een volledig nieuwe callable maken maar kunnen we gewoon de parameter aanpassen
DefaultConv2D(use_bias=True)

In [ ]:

# ResNet Code komt uit het boek

DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

# Class ResidualUnit is afgeleid van keras.layers.Layer -> alle methodes van Layer kunnen we ook gebruiken
class ResidualUnit(keras.layers.Layer):
    # Maakt lagen aan, voeren geen berekeningen uit
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        # Lijst van instanties die we als lagen willen gebruiken
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()
        ]
        # Als stride 1 is, is skip laag leeg (cfr spatiale dimensies van input worden niet aangepast)
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()
            ]

    # voert berekeningen uit op tensoren
    def call(self, inputs):
        # tensor van batch van inputs
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        # skip connectie -> activatie functie op de plus knoop (waar Z en skip-Z opgeteld worden)
        return self.activation(Z + skip_Z)



In [ ]:
model = keras.Sequential([
    DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[224, 224, 3]), # stride deelt spatiale afmetingen door 2
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"), # pooling deelt spatiale afmetingen nog eens door 2 -> 4
])
prev_filters = 64
# residual blocks toevoegen:
# Bij ResNet 34 is dat: 3 met 64 filters, 4 met 128 filters, 6 met 256 filters en 3 met 512 filters
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    # kijken of we veranderen van aantal filters -> andere stride nodig -> skiplaag met een conv2d of niet
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))
